### Notes from watching dozens of YouTube videos

* Docker is a software development platform and a virtualization technology; key concept is hermetic operation, no shock waves into the system at large
    * apps can run universally / system agnostic
    * Docker communicates "natively" (directly) with the system kernel
    * Multiple Docker containers running on a single system from a common source image will share files to reduce total footprint
* Replicate an environment on any machine: Covers scale and consistency: Run many, anywhere.
* VMs can be bulky and slow. Containers only virtualize the OS, not the hardware
* Three fundamental elements
    - Docker File is like DNA: Code: How to build an image 
        - Start with `FROM` to "pull down" a base image
        - Next a `RUN` command to `apt-get install sl` (which is a little text-based train)
        - Next a `ENV` command to set `PORT=8080` environment variables used inside the running Container
        - Finally a `CMD` default command with `["echo", "Hello Docker"]` some command-y sort of run-time task
    - Docker Image is an immutable file: A snapshot of software + dependencies all the way down to the OS level: configuration content: Spin up container
    - Docker Container is a running-then-halting program
    
    
As we work in the Docker headspace let's be aware that...
- a `Dockerfile` lives in a host folder with an appropriate name
- a Docker *image* lives for all we know on the planet Neptune; it is not *visible* as a normal file is... so this is being managed behind the scenes
- the `docker` command is the **command line interface** that interfaces with the **docker daemon** that in turn manages the local docker landscape
- all of this is put into place when we install Docker Desktop (an app)

In [9]:
# Let's build a Dockerfile using Python

def WriteDockerFile(s, f):
    with open(f, "w") as file: file.write(s)


def AddFROM(s, os):
    s += 'FROM ' + os + '\n\n'
    return s

    
def AddUSER(s, who):
    s += 'USER ' + who + '\n\n'
    return s

def AddRUN(s, pkgs):
    for r in pkgs:
        s += 'RUN ' + r[0] + ' ' + r[1] + '\n'
    s += '\n'
    return s

def AddENV(s, envs):
    for e in envs:
        s += 'ENV ' + e[0] + '=' + e[1] + '\n'
    s += '\n'
    return s

def AddCMD(s, cmd):
    s += 'CMD ["' + cmd[0] + '", "' + cmd[1] + '"]'
    s += '\n\n'
    return s


d = ''
d = AddFROM(d, 'ubuntu:20.04')
d = AddUSER(d, 'root')
# d = AddRUN(d, [['apt-get install -V', 'sl']])         # '-y curl' is a common install
d = AddENV(d, [['PORT', '8080']])
d = AddCMD(d, ['echo', 'Hello Docker'])

print(d)

# make sure this target directory exists!
WriteDockerFile(d, "/home/kilroy/docker/Dockerfile")

FROM ubuntu:20.04

USER root

ENV PORT=8080

CMD ["echo", "Hello Docker"]




### Cautionary remark


For the next cell to run: I must start Docker Desktop on my PC; which makes the `docker` command available in WSL2.

In [13]:
!docker build -t train /home/kilroy/docker 
# --progress=quiet

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 118B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/ubuntu:20.04            0.0s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 118B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for dock

In [14]:
# use docker image list or equivalently:
!docker images


The command 'docker' could not be found in this WSL 2 distro.
We recommend to activate the WSL integration in Docker Desktop settings.

For details about using Docker Desktop with WSL 2, visit:

https://docs.docker.com/go/wsl2/



In [ ]:
# are any docker images running on this machine at the moment?
!docker ps

### Observe


The output is `sha256:7a3df4ee0998d0a8ccdd53b56e9063c455b21283c177e38366fa985c379ca96c`... meaning what?
Notice the first 12 characters after the `:` corresponds to the `IMAGE ID`.


### Key questions


- Now that a Docker app -- i.e. an *image* -- called `train` exists: How does the `docker run train` command know where to find that image?
- If I run `train` or `hello-world` from this notebook or `bash` I get the expected message **Hello Docker** etc... what if I run `ubuntu`?
    - Answer: `ubuntu` does not stick; I'm probably missing some detail. See the list / status in the Docker app



In [ ]:
# use --rm to clean up when done; and -it to run interactively
!docker run --rm -it train

In [ ]:
!docker run --rm -it hello-world

In [ ]:
# This command by virtue of -it keeps running. It can be halted from the Docker App
# !docker run --rm -it ubuntu

### Pivot to from running a container to Docker hub


The bash command `docker run ubuntu` is supposed to place us in an Ubuntu `bash` shell but it just starts/halts. 


`docker pull` will get an existing image from DockerHub, here without then with a tag. No tag: Latest.


```
docker pull ubuntu
docker pull ubuntu:20.04
docker pull naclomi/textbook-writer
```

The `textbook-writer` takes a couple minutes to pull. Once that completes we have...

In [ ]:
!docker images

In [ ]:
!docker run --rm -it naclomi/textbook-writer

### ok great now i should do one too


This was so encouraging I thought I'd check in at [DockerHub](https://hub.docker.com) and create a repo and push an image.
On my development laptop these are the key docker commands: 

```
docker tag local-image:tagname new-repo:tagname
docker push new-repo:tagname
```


My Docker image will be called `calcul8`. To begin as simply as possible my code will take two environment variables
a and b and calculate all the primes on [a, b).